In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import datetime
from typing import Optional

%matplotlib inline

## Gather Data

In [3]:
def get_dataframe_from_sql(query, db_path = 'football.sqlite'):
    """
    Returns a pandas dataframe containing the db data returned
    by the provided SQL query.
    """
    # establish a connection to the database
    conn = sqlite3.connect(db_path)

    # load the query results into a pandas dataframe
    df = pd.read_sql_query(query, conn)

    # close the connection to the database
    conn.close()

    # return the dataframe
    return df

In [4]:
get_dataframe_from_sql("SELECT name FROM sqlite_master WHERE type='table';")

,name
0,sqlite_sequence
1,Player_Attributes
2,Player
3,Match
4,League
5,Country
6,Team
7,Team_Attributes


In [5]:
get_dataframe_from_sql("PRAGMA table_info(Player_Attributes);").head(30)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,player_fifa_api_id,INTEGER,0,None,0
2,2,player_api_id,INTEGER,0,None,0
3,3,date,TEXT,0,None,0
4,4,overall_rating,INTEGER,0,None,0
5,5,potential,INTEGER,0,None,0
6,6,preferred_foot,TEXT,0,None,0
7,7,attacking_work_rate,TEXT,0,None,0
8,8,defensive_work_rate,TEXT,0,None,0
9,9,crossing,INTEGER,0,None,0


In [6]:
get_dataframe_from_sql("SELECT * FROM Team;").head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


#### get the top 5 leagues

In [7]:
top_5_leagues_names = {
    "England Premier League",
    "France Ligue 1",
    "Germany 1. Bundesliga",
    "Italy Serie A",
    "Spain LIGA BBVA",
}

league_name_values = [f"\'{name}\'" for name in top_5_leagues_names]
leagues_df = get_dataframe_from_sql(f'SELECT * FROM League WHERE name IN ({", ".join(league_name_values)})')
leagues_df

,id,country_id,name
0,1729,1729,England Premier League
1,4769,4769,France Ligue 1
2,7809,7809,Germany 1. Bundesliga
3,10257,10257,Italy Serie A
4,21518,21518,Spain LIGA BBVA


#### filter the matches that are in each league

In [8]:
top_5_leagues_matches = get_dataframe_from_sql('SELECT * FROM Match').query('league_id in @leagues_df.id')
top_5_leagues_matches = top_5_leagues_matches.sort_values("date").reset_index(drop=True)
top_5_leagues_matches.sample(10)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
3345,10903,10257,10257,2009/2010,33,2010-04-10 00:00:00,705396,8535,8636,2,...,2.20,3.50,3.20,2.10,3.50,3.25,2.05,3.40,3.30,2.10
8603,6493,4769,4769,2012/2013,28,2013-03-10 00:00:00,1217085,9748,8592,0,...,3.30,2.25,3.40,3.50,2.10,3.30,3.50,2.10,3.20,3.50
9180,6896,4769,4769,2013/2014,3,2013-08-24 00:00:00,1468186,10249,9874,2,...,4.80,1.80,3.60,5.00,NaN,NaN,NaN,NaN,NaN,NaN
3005,22048,21518,21518,2009/2010,23,2010-02-21 00:00:00,684941,9864,8558,2,...,3.75,2.10,3.20,3.50,2.10,3.25,3.40,2.10,3.25,3.50
70,10266,10257,10257,2008/2009,1,2008-08-30 00:00:00,537641,8600,8540,3,...,4.00,2.00,3.00,3.50,2.10,3.00,3.60,2.00,3.00,3.60
12347,7294,4769,4769,2014/2015,31,2015-04-04 00:00:00,1710002,8550,9941,3,...,NaN,2.88,3.13,2.80,NaN,NaN,NaN,NaN,NaN,NaN
8828,9248,7809,7809,2012/2013,30,2013-04-20 00:00:00,1239737,9790,8194,2,...,4.33,1.85,3.75,4.60,1.83,3.60,4.20,1.83,3.50,4.33
1228,10445,10257,10257,2008/2009,26,2009-03-01 00:00:00,539817,8529,9804,0,...,4.33,1.91,3.20,3.75,2.05,3.05,4.00,1.95,3.10,4.00
3692,5754,4769,4769,2010/2011,3,2010-08-21 00:00:00,829894,9831,8481,1,...,3.10,2.30,3.00,3.12,2.25,3.00,3.20,2.25,3.10,3.10
7609,9312,7809,7809,2012/2013,7,2012-10-06 00:00:00,1239521,9823,8226,2,...,15.00,1.15,8.50,19.00,1.15,7.50,15.00,1.14,7.50,15.00


In [9]:
top_5_leagues_matches.loc[:, list(top_5_leagues_matches.columns[-30:-15])].describe()

,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA
count,14573.000000,14573.000000,14573.000000,14558.000000,14558.000000,14558.000000,14540.000000,14540.000000,14540.000000,14568.000000,14568.000000,14568.000000,7292.000000,7292.000000,7292.000000
mean,2.595606,3.809271,4.704510,2.541709,3.738281,4.455709,2.454475,3.625189,4.193165,2.512505,3.706167,4.429821,2.787401,4.140014,5.128762
std,1.756999,1.171708,3.794316,1.625401,1.090557,3.359353,1.434216,0.838685,2.881999,1.606546,1.069737,3.447060,2.191845,1.680150,4.701423
min,1.040000,1.400000,1.080000,1.030000,1.650000,1.100000,1.050000,1.500000,1.100000,1.040000,1.400000,1.100000,1.040000,2.200000,1.090000
25%,1.670000,3.250000,2.600000,1.700000,3.200000,2.600000,1.700000,3.200000,2.550000,1.670000,3.200000,2.500000,1.710000,3.380000,2.617500
50%,2.100000,3.400000,3.600000,2.100000,3.400000,3.450000,2.100000,3.300000,3.300000,2.100000,3.400000,3.400000,2.180000,3.600000,3.730000
75%,2.750000,3.800000,5.250000,2.700000,3.800000,5.000000,2.600000,3.700000,4.650000,2.700000,3.750000,5.000000,2.940000,4.180000,5.550000
max,26.000000,17.000000,51.000000,34.000000,19.500000,51.000000,20.000000,11.000000,25.000000,26.000000,19.000000,51.000000,36.000000,29.000000,47.500000


In [10]:
top_5_leagues_matches.loc[:, list(top_5_leagues_matches.columns[-15:])].describe()

,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
count,14568.000000,14568.000000,14568.000000,11074.000000,11074.000000,11074.000000,14555.000000,14555.000000,14555.000000,9081.000000,9081.000000,9081.000000,9085.000000,9085.000000,9085.000000
mean,2.569285,3.629590,4.570721,2.533537,3.736561,4.680812,2.642940,3.875681,4.909706,2.472426,3.622296,4.386528,2.465726,3.625150,4.418149
std,1.688519,0.987731,3.745467,1.627928,1.036067,3.716216,1.916398,1.325661,4.465039,1.454708,0.883999,2.998704,1.460544,0.871147,3.158631
min,1.020000,1.020000,1.080000,1.040000,1.400000,1.100000,1.030000,1.620000,1.080000,1.050000,1.450000,1.120000,1.040000,1.330000,1.120000
25%,1.700000,3.200000,2.620000,1.670000,3.250000,2.630000,1.700000,3.250000,2.620000,1.700000,3.200000,2.600000,1.670000,3.200000,2.620000
50%,2.150000,3.300000,3.400000,2.100000,3.400000,3.600000,2.150000,3.500000,3.600000,2.100000,3.300000,3.500000,2.100000,3.300000,3.400000
75%,2.700000,3.750000,5.000000,2.700000,3.750000,5.250000,2.800000,4.000000,5.400000,2.630000,3.700000,5.000000,2.600000,3.750000,5.000000
max,26.000000,17.000000,51.000000,23.000000,15.000000,41.000000,36.000000,26.000000,67.000000,21.000000,11.000000,34.000000,17.000000,13.000000,34.000000


In [11]:
top_5_leagues_matches.loc[:, list(top_5_leagues_matches.columns[-30:-15])].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14585 entries, 0 to 14584
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   B365H   14573 non-null  float64
 1   B365D   14573 non-null  float64
 2   B365A   14573 non-null  float64
 3   BWH     14558 non-null  float64
 4   BWD     14558 non-null  float64
 5   BWA     14558 non-null  float64
 6   IWH     14540 non-null  float64
 7   IWD     14540 non-null  float64
 8   IWA     14540 non-null  float64
 9   LBH     14568 non-null  float64
 10  LBD     14568 non-null  float64
 11  LBA     14568 non-null  float64
 12  PSH     7292 non-null   float64
 13  PSD     7292 non-null   float64
 14  PSA     7292 non-null   float64
dtypes: float64(15)
memory usage: 1.7 MB


In [12]:
top_5_leagues_matches.loc[:, list(top_5_leagues_matches.columns[-15:])].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14585 entries, 0 to 14584
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   WHH     14568 non-null  float64
 1   WHD     14568 non-null  float64
 2   WHA     14568 non-null  float64
 3   SJH     11074 non-null  float64
 4   SJD     11074 non-null  float64
 5   SJA     11074 non-null  float64
 6   VCH     14555 non-null  float64
 7   VCD     14555 non-null  float64
 8   VCA     14555 non-null  float64
 9   GBH     9081 non-null   float64
 10  GBD     9081 non-null   float64
 11  GBA     9081 non-null   float64
 12  BSH     9085 non-null   float64
 13  BSD     9085 non-null   float64
 14  BSA     9085 non-null   float64
dtypes: float64(15)
memory usage: 1.7 MB


### get the player attributes

In [13]:
player_attributes = get_dataframe_from_sql('SELECT * FROM Player_Attributes;')
player_attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


## Wrangle Data

#### handling missing betting predictions
- drop columns that have less than 10k non-null predictions (that's less than 2/3 of the data and that feels too incomplete to me)
    - DONE!
- drop all match rows that have no predictions at all
- fill in the remaining null predictions using the average of those values for the row (i.e. null home predictions will be average of home predictions, likewise for draw and away)

##### drop predictions columns that have less than 10k non-null predictions (that's less than 2/3 of the data and that feels too incomplete to me)

In [14]:
num_missing_predictions = top_5_leagues_matches.isna().sum().iloc[-30:]
num_missing_predictions

B365H      12
B365D      12
B365A      12
BWH        27
BWD        27
BWA        27
IWH        45
IWD        45
IWA        45
LBH        17
LBD        17
LBA        17
PSH      7293
PSD      7293
PSA      7293
WHH        17
WHD        17
WHA        17
SJH      3511
SJD      3511
SJA      3511
VCH        30
VCD        30
VCA        30
GBH      5504
GBD      5504
GBA      5504
BSH      5500
BSD      5500
BSA      5500
dtype: int64

In [15]:
# drop the columns for the betting providers with more than 1/3 of the data missing
top_5_leagues_matches.drop(num_missing_predictions[num_missing_predictions > round(top_5_leagues_matches.shape[0] * 1 / 3)].index, axis=1, inplace=True)
top_5_leagues_matches.columns

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'LBA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA'],
      dtype='object', length=106)

In [33]:
match_predictions_df = top_5_leagues_matches.loc[:, list(top_5_leagues_matches.columns[-21:])]
match_predictions_df.columns

Index(['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
       'LBH', 'LBD', 'LBA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH',
       'VCD', 'VCA'],
      dtype='object')

In [40]:
match_predictions_df.iloc[:, :-6].describe()

,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,WHH,WHD,WHA
count,14573.000000,14573.000000,14573.000000,14558.000000,14558.000000,14558.000000,14540.000000,14540.000000,14540.000000,14568.000000,14568.000000,14568.000000,14568.000000,14568.000000,14568.000000
mean,2.595606,3.809271,4.704510,2.541709,3.738281,4.455709,2.454475,3.625189,4.193165,2.512505,3.706167,4.429821,2.569285,3.629590,4.570721
std,1.756999,1.171708,3.794316,1.625401,1.090557,3.359353,1.434216,0.838685,2.881999,1.606546,1.069737,3.447060,1.688519,0.987731,3.745467
min,1.040000,1.400000,1.080000,1.030000,1.650000,1.100000,1.050000,1.500000,1.100000,1.040000,1.400000,1.100000,1.020000,1.020000,1.080000
25%,1.670000,3.250000,2.600000,1.700000,3.200000,2.600000,1.700000,3.200000,2.550000,1.670000,3.200000,2.500000,1.700000,3.200000,2.620000
50%,2.100000,3.400000,3.600000,2.100000,3.400000,3.450000,2.100000,3.300000,3.300000,2.100000,3.400000,3.400000,2.150000,3.300000,3.400000
75%,2.750000,3.800000,5.250000,2.700000,3.800000,5.000000,2.600000,3.700000,4.650000,2.700000,3.750000,5.000000,2.700000,3.750000,5.000000
max,26.000000,17.000000,51.000000,34.000000,19.500000,51.000000,20.000000,11.000000,25.000000,26.000000,19.000000,51.000000,26.000000,17.000000,51.000000


In [41]:
match_predictions_df.iloc[:, -6:].describe()

,SJH,SJD,SJA,VCH,VCD,VCA
count,11074.000000,11074.000000,11074.000000,14555.000000,14555.000000,14555.000000
mean,2.533537,3.736561,4.680812,2.642940,3.875681,4.909706
std,1.627928,1.036067,3.716216,1.916398,1.325661,4.465039
min,1.040000,1.400000,1.100000,1.030000,1.620000,1.080000
25%,1.670000,3.250000,2.630000,1.700000,3.250000,2.620000
50%,2.100000,3.400000,3.600000,2.150000,3.500000,3.600000
75%,2.700000,3.750000,5.250000,2.800000,4.000000,5.400000
max,23.000000,15.000000,41.000000,36.000000,26.000000,67.000000


##### drop all match rows that have no predictions at all

In [29]:
top_5_leagues_matches.shape

(14585, 106)

In [42]:
top_5_leagues_matches.drop(top_5_leagues_matches[match_predictions_df.iloc[:, -21:].isna().sum(axis=1) == 21].index, inplace=True)
top_5_leagues_matches.shape

(14576, 106)

##### fill in the remaining null predictions using the average of those values for the row (i.e. null home predictions will be average of home predictions, likewise for draw and away)

In [50]:
# strategy: get the average home odds for the row, and fill in with that value
# fillna allows me to specify a dataframe with values for each column to fill in with
# so I can specify a dataframe with the average home odds for each row, in the home column only
# do the same for the away and draw columns
home_predictions_cols = list(filter(lambda x: x.endswith("H"), match_predictions_df.columns))
# match_predictions_df[home_predictions_cols] = match_predictions_df[home_predictions_cols].fillna(match_predictions_df[home_predictions_cols].mean(axis=1).to_frame())
# TODO: almost there! The means are being gotten correctly but the fillna is not working because the columns are not being matched up correctly
# the solution should be put the means in a dataframe with the same columns as the match_predictions_df, and with the columns containing the same values (the means)
match_predictions_df[home_predictions_cols].fillna(match_predictions_df[home_predictions_cols].mean(axis=1))

,B365H,BWH,IWH,LBH,WHH,SJH,VCH
0,1.57,1.55,1.50,1.53,1.53,1.5,1.60
1,2.30,2.40,2.30,2.37,2.30,2.4,2.30
2,2.10,1.95,1.90,2.00,2.00,2.0,2.00
3,2.40,2.40,2.30,2.25,2.30,2.4,2.25
4,2.15,2.25,2.20,2.10,2.15,2.1,2.10
...,...,...,...,...,...,...,...
14580,2.00,2.00,1.85,1.95,2.05,NaN,2.05
14581,1.80,1.83,1.85,1.80,1.83,NaN,1.85
14582,1.33,1.33,1.40,1.33,1.29,NaN,1.33
14583,3.50,3.30,2.90,3.40,3.50,NaN,3.50


In [43]:
match_predictions_df.columns

Index(['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
       'LBH', 'LBD', 'LBA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH',
       'VCD', 'VCA'],
      dtype='object')

In [ ]:
top_5_leagues_matches.loc[:, 'home_team_goal':'away_team_goal'].describe()

,home_team_goal,away_team_goal
count,14585.000000,14585.000000
mean,1.539184,1.140075
std,1.289485,1.129409
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,10.000000,9.000000


In [130]:
season_categories = pd.CategoricalDtype(top_5_leagues_matches.season.unique(), ordered=True)
season_categories

CategoricalDtype(categories=['2008/2009', '2009/2010', '2010/2011', '2011/2012',
                  '2012/2013', '2013/2014', '2014/2015', '2015/2016'],
, ordered=True)

In [131]:
# convert the season column for matches to this categorical variable
top_5_leagues_matches["season"] = top_5_leagues_matches.season.astype(season_categories)
top_5_leagues_matches.season.dtypes

CategoricalDtype(categories=['2008/2009', '2009/2010', '2010/2011', '2011/2012',
                  '2012/2013', '2013/2014', '2014/2015', '2015/2016'],
, ordered=True)

In [132]:
# convert the date strings to date time objects
player_attributes["date"] = pd.to_datetime(player_attributes.date)
player_attributes.date.head()

0   2016-02-18
1   2015-11-19
2   2015-09-21
3   2015-03-20
4   2007-02-22
Name: date, dtype: datetime64[ns]

In [133]:
season_categories

CategoricalDtype(categories=['2008/2009', '2009/2010', '2010/2011', '2011/2012',
                  '2012/2013', '2013/2014', '2014/2015', '2015/2016'],
, ordered=True)

In [134]:
player_attributes.date.min()

Timestamp('2007-02-22 00:00:00')

In [135]:
player_attributes.date.max()

Timestamp('2016-07-07 00:00:00')

In [136]:
# add a season column to the player attributes df based on the date column
def find_season(items, condition):
    for item in items:
        if condition(item):
            return item
    return None  # Return None if no match is found


def get_season_from_date(date: datetime.datetime) -> Optional[pd.CategoricalDtype]:
    """
    Returns the season category for the supplied date
    """
    july = 7
    if date.year < 2008 or (date.year == 2008 and date.month <= july):
        return None

    return find_season(season_categories.categories, lambda season: (f"{date.year}/" in season and date.month > july) or (f"/{date.year}" in season and date.month <= july))

In [137]:
player_attributes.date.head()

0   2016-02-18
1   2015-11-19
2   2015-09-21
3   2015-03-20
4   2007-02-22
Name: date, dtype: datetime64[ns]

In [138]:
player_attributes["season"] = player_attributes.date.apply(get_season_from_date).astype(season_categories)
player_attributes.season.head()

0    2015/2016
1    2015/2016
2    2015/2016
3    2014/2015
4          NaN
Name: season, dtype: category
Categories (8, object): ['2008/2009' < '2009/2010' < '2010/2011' < '2011/2012' < '2012/2013' < '2013/2014' < '2014/2015' < '2015/2016']

In [139]:
# Reshape the matches dataframe to have a single column for player IDs
home_player_cols = [f'home_player_{i}' for i in range(1, 12)]
away_player_cols = [f'away_player_{i}' for i in range(1, 12)]
player_api_ids = pd.concat([top_5_leagues_matches[home_player_cols], top_5_leagues_matches[away_player_cols]], axis=1).stack().reset_index(drop=True)
player_api_ids = player_api_ids.rename('player_api_id')

player_api_ids.head()

0    30458.0
1    32571.0
2    38703.0
3    40543.0
4    26119.0
Name: player_api_id, dtype: float64

In [140]:
# remove the players that do not feature in any match in the top 5 leagues over the seasons
player_attributes = player_attributes[player_attributes.player_api_id.isin(player_api_ids)]
player_attributes.shape

(121656, 43)

In [141]:
top_5_leagues_matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,4770,4769,4769,2008/2009,1,2008-08-09 00:00:00,483130,9827,7819,2,...,7.5,1.60,3.3,5.50,1.57,3.6,6.00,1.50,3.50,6.50
1,4771,4769,4769,2008/2009,1,2008-08-09 00:00:00,483131,9746,9831,1,...,3.1,2.30,2.9,3.00,2.35,2.9,3.20,2.38,2.88,2.88
2,4772,4769,4769,2008/2009,1,2008-08-09 00:00:00,483132,8682,8689,0,...,4.0,2.00,2.9,3.75,2.00,3.1,3.75,1.91,3.00,4.00
3,4774,4769,4769,2008/2009,1,2008-08-09 00:00:00,483134,9829,9847,1,...,3.2,2.25,3.0,3.00,2.38,3.0,3.00,2.30,3.00,2.88
4,4775,4769,4769,2008/2009,1,2008-08-09 00:00:00,483135,8481,8639,0,...,3.8,2.10,3.0,3.30,2.20,3.0,3.30,2.00,3.00,3.60


## another strategy
- sort the matches by date
- for each row in player attributes
    - find the first row in player matches that has both the season and the player ID of the player attributes row
    - if there is, return its league id
    - else, return `None`
- I therefore need the player matches table
    - it has a column for player ID and match ID
    - each match ID from the matches table gets 22 rows, one for each player that started the match
    - each match id from the matches table has the season in which the match was played
    - each match id from the matches table has the league to which the match belongs

In [142]:
top_5_leagues_matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,4770,4769,4769,2008/2009,1,2008-08-09 00:00:00,483130,9827,7819,2,...,7.5,1.60,3.3,5.50,1.57,3.6,6.00,1.50,3.50,6.50
1,4771,4769,4769,2008/2009,1,2008-08-09 00:00:00,483131,9746,9831,1,...,3.1,2.30,2.9,3.00,2.35,2.9,3.20,2.38,2.88,2.88
2,4772,4769,4769,2008/2009,1,2008-08-09 00:00:00,483132,8682,8689,0,...,4.0,2.00,2.9,3.75,2.00,3.1,3.75,1.91,3.00,4.00
3,4774,4769,4769,2008/2009,1,2008-08-09 00:00:00,483134,9829,9847,1,...,3.2,2.25,3.0,3.00,2.38,3.0,3.00,2.30,3.00,2.88
4,4775,4769,4769,2008/2009,1,2008-08-09 00:00:00,483135,8481,8639,0,...,3.8,2.10,3.0,3.30,2.20,3.0,3.30,2.00,3.00,3.60


In [143]:
player_matches = pd.melt(top_5_leagues_matches, id_vars=["id"], value_vars=home_player_cols+away_player_cols, value_name="player_api_id").drop("variable", axis=1)
# player_matches.rename({"id": "match_id"}, axis=1, inplace=True)
# player_matches.sort_values("match_id", inplace=True)
player_matches

,id,player_api_id
0,4770,30458.0
1,4771,41186.0
2,4772,NaN
3,4774,145039.0
4,4775,26233.0
...,...,...
320865,24492,200917.0
320866,24495,179083.0
320867,24497,25462.0
320868,4705,47382.0


In [144]:
league_matches = pd.melt(top_5_leagues_matches, id_vars=["league_id"], value_vars=home_player_cols+away_player_cols, value_name="player_api_id").drop("variable", axis=1)
# league_matches.rename({"id": "match_id"}, axis=1, inplace=True)
# league_matches.sort_values("match_id", inplace=True)
league_matches

,league_id,player_api_id
0,4769,30458.0
1,4769,41186.0
2,4769,NaN
3,4769,145039.0
4,4769,26233.0
...,...,...
320865,21518,200917.0
320866,21518,179083.0
320867,21518,25462.0
320868,1729,47382.0


In [145]:
season_matches = pd.melt(top_5_leagues_matches, id_vars=["season"], value_vars=home_player_cols+away_player_cols, value_name="player_api_id").drop("variable", axis=1)
# season_matches.rename({"id": "match_id"}, axis=1, inplace=True)
# season_matches.sort_values("match_id", inplace=True)
season_matches

,season,player_api_id
0,2008/2009,30458.0
1,2008/2009,41186.0
2,2008/2009,NaN
3,2008/2009,145039.0
4,2008/2009,26233.0
...,...,...
320865,2015/2016,200917.0
320866,2015/2016,179083.0
320867,2015/2016,25462.0
320868,2015/2016,47382.0


In [146]:
player_matches = player_matches.merge(league_matches, left_index=True, right_index=True).merge(season_matches, left_index=True, right_index=True).drop(labels=["player_api_id_x", "player_api_id_y"], axis=1)
player_matches.head()

,id,league_id,season,player_api_id
0,4770,4769,2008/2009,30458.0
1,4771,4769,2008/2009,41186.0
2,4772,4769,2008/2009,NaN
3,4774,4769,2008/2009,145039.0
4,4775,4769,2008/2009,26233.0


In [167]:
# preserve only the first occurrence of a player in a season
player_matches = player_matches[~(player_matches[['player_api_id', 'season']].duplicated())]
player_matches.head()

,id,league_id,season,player_api_id
0,4770,4769,2008/2009,30458.0
1,4771,4769,2008/2009,41186.0
2,4772,4769,2008/2009,NaN
3,4774,4769,2008/2009,145039.0
4,4775,4769,2008/2009,26233.0


In [168]:
player_attributes = pd.merge(player_attributes, player_matches, left_on=['player_api_id', 'season'], right_on=['player_api_id', 'season'], how='left')
player_attributes

,id_x,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,season,id_y,league_id
0,6,189615,155782,2016-04-21,74.0,76.0,left,high,medium,80.0,...,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,4595.0,1729.0
1,7,189615,155782,2016-04-07,74.0,76.0,left,high,medium,80.0,...,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,4595.0,1729.0
2,8,189615,155782,2016-01-07,73.0,75.0,left,high,medium,79.0,...,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,4595.0,1729.0
3,9,189615,155782,2015-12-24,73.0,75.0,left,high,medium,79.0,...,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,4595.0,1729.0
4,10,189615,155782,2015-12-17,73.0,75.0,left,high,medium,79.0,...,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,4595.0,1729.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121651,183974,102359,39902,2009-08-30,83.0,85.0,right,medium,low,84.0,...,31.0,30.0,9.0,20.0,84.0,20.0,20.0,2009/2010,8115.0,7809.0
121652,183975,102359,39902,2009-02-22,78.0,80.0,right,medium,low,74.0,...,31.0,30.0,9.0,20.0,73.0,20.0,20.0,2008/2009,8100.0,7809.0
121653,183976,102359,39902,2008-08-30,77.0,80.0,right,medium,low,74.0,...,31.0,30.0,9.0,20.0,73.0,20.0,20.0,2008/2009,8100.0,7809.0
121654,183977,102359,39902,2007-08-30,78.0,81.0,right,medium,low,74.0,...,32.0,30.0,9.0,20.0,73.0,20.0,20.0,NaN,NaN,NaN


In [172]:
player_attributes = player_attributes[~(player_attributes.league_id.isna())]
player_attributes.reset_index(drop=True, inplace=True)
player_attributes

,id_x,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,season,id_y,league_id
0,6,189615,155782,2016-04-21,74.0,76.0,left,high,medium,80.0,...,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,4595.0,1729.0
1,7,189615,155782,2016-04-07,74.0,76.0,left,high,medium,80.0,...,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,4595.0,1729.0
2,8,189615,155782,2016-01-07,73.0,75.0,left,high,medium,79.0,...,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,4595.0,1729.0
3,9,189615,155782,2015-12-24,73.0,75.0,left,high,medium,79.0,...,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,4595.0,1729.0
4,10,189615,155782,2015-12-17,73.0,75.0,left,high,medium,79.0,...,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,4595.0,1729.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61641,183965,47058,35506,2009-02-22,70.0,78.0,right,None,_0,48.0,...,74.0,65.0,14.0,25.0,51.0,25.0,25.0,2008/2009,1765.0,1729.0
61642,183966,47058,35506,2008-08-30,72.0,78.0,right,None,_0,48.0,...,78.0,65.0,14.0,25.0,51.0,25.0,25.0,2008/2009,1765.0,1729.0
61643,183974,102359,39902,2009-08-30,83.0,85.0,right,medium,low,84.0,...,31.0,30.0,9.0,20.0,84.0,20.0,20.0,2009/2010,8115.0,7809.0
61644,183975,102359,39902,2009-02-22,78.0,80.0,right,medium,low,74.0,...,31.0,30.0,9.0,20.0,73.0,20.0,20.0,2008/2009,8100.0,7809.0


In [177]:
player_attributes = player_attributes.rename({"id_x": "id"}, axis=1).drop("id_y", axis=1)
player_attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,season,league_id
0,6,189615,155782,2016-04-21,74.0,76.0,left,high,medium,80.0,...,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,1729.0
1,7,189615,155782,2016-04-07,74.0,76.0,left,high,medium,80.0,...,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,1729.0
2,8,189615,155782,2016-01-07,73.0,75.0,left,high,medium,79.0,...,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,1729.0
3,9,189615,155782,2015-12-24,73.0,75.0,left,high,medium,79.0,...,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,1729.0
4,10,189615,155782,2015-12-17,73.0,75.0,left,high,medium,79.0,...,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,2015/2016,1729.0
